In [121]:
# Main import block
import asyncio
import datetime
import signal
import time
import warnings
from contextlib import contextmanager
from datetime import date, datetime
from io import StringIO
import os

# Sneaky scrapper
import cfscrape
# Upload to google sheets
import gspread
# import df2gspread as d2g
import gspread_pandas as gsp
import numpy as np
import pandas as pd
import httpx
import requests
# Upload to yandex
import yadisk
from bs4 import BeautifulSoup
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

warnings.filterwarnings("ignore")

In [ ]:
################################################################
################## NEW_WESTMETALL###############################
################################################################


async def new_westmetall_async():
    async def get_data(metall, col_name):
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        response = requests.get(url=url)

        df = pd.read_html(response.text)[0][:30]
        data = df.iloc[:, :2]

        data = data.query("date != 'date'")
        data["date"] = pd.to_datetime(data["date"])
        data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
        data.rename(columns={data.columns[1]: col_name}, inplace=True)

        return data

    try:
        al = await get_data(metall="Al", col_name="aluminium")
        cu = await get_data(metall="Cu", col_name="copper")
        pb = await get_data(metall="Pb", col_name="lead")
        ni = await get_data(metall="Ni", col_name="nickel")
        zn = await get_data(metall="Zn", col_name="zink")
        tn = await get_data(metall="Sn", col_name="tin")

        result = (
            pd.merge(al, cu, on="date", how="left")
            .merge(pb, on="date", how="left")
            .merge(ni, on="date", how="left")
            .merge(zn, on="date", how="left")
            .merge(tn, on="date", how="left")
        )

        old_data = pd.read_excel("./data/LME_westmetall_db.xlsx", index_col=0)

        final_data = pd.concat([old_data, result], axis=0)

        final_data.drop_duplicates(subset="date", inplace=True)

        final_data.sort_values(by="date", inplace=True)

        final_data.reset_index(inplace=True, drop=True)

        with pd.ExcelWriter(
            "./data/LME_westmetall_db.xlsx",
            date_format="YYYY-MM-DD",
            datetime_format="YYYY-MM-DD",
        ) as writer:
            final_data.to_excel(writer, sheet_name="LME_westmetall")

        print("WESTMETALL is done!!!")
        # display(final_data.tail(10))

    except Exception as error:
        print(error)


In [20]:
url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Al_cash"
response = requests.get(url=url)

In [66]:
url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Cu_cash"
response = requests.get(url=url)

In [ ]:
#Tin
url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Sn_cash"
response = requests.get(url=url)

In [ ]:
# Lead
url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Pb_cash"
response = requests.get(url=url)

In [99]:
# Zink
url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Zn_cash"
response = requests.get(url=url)

In [110]:
# Nickel
url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_Ni_cash"
response = requests.get(url=url)

In [111]:
df = pd.read_html(response.text)
#data = df.iloc[:, :4]

In [112]:
data = pd.DataFrame()

In [113]:
for i in range(len(df)):
    data = pd.concat([data, df[i]], axis=0)

In [114]:
data = data[data["date"] != 'date']

In [115]:
data = data[data.iloc[:, 1] != '-']
data = data[data.iloc[:, 2] != '-']
data = data[data.iloc[:, 3] != '-']

In [116]:
data_preprocessed = data.copy()

In [117]:
data_preprocessed["date"] = pd.to_datetime(data_preprocessed["date"])
data_preprocessed.iloc[:, 1] = pd.to_numeric(data_preprocessed.iloc[:, 1])
data_preprocessed.iloc[:, 2] = pd.to_numeric(data_preprocessed.iloc[:, 2])

In [118]:
data_preprocessed

,date,LME Nickel Cash-Settlement,LME Nickel 3-month,LME Nickel stock
0,2025-03-07,16050.0,16270.0,198522
1,2025-03-06,15990.0,16180.0,199410
2,2025-03-05,15675.0,15975.0,194520
3,2025-03-04,15760.0,15970.0,195360
4,2025-03-03,15600.0,15770.0,195162
...,...,...,...,...
261,2008-01-08,28855.0,29135.0,48156
262,2008-01-07,27905.0,28205.0,48126
263,2008-01-04,29515.0,29825.0,47784
264,2008-01-03,28105.0,28400.0,47892


In [119]:
data_preprocessed.to_csv("./data/nickel_database.csv", index=False)

In [124]:
pd.read_csv("./data/nickel_database.csv")

,date,LME Nickel Cash-Settlement,LME Nickel 3-month,LME Nickel stock
0,2025-03-07 00:00:00,16050.0,16270.0,198522
1,2025-03-06 00:00:00,15990.0,16180.0,199410
2,2025-03-05 00:00:00,15675.0,15975.0,194520
3,2025-03-04 00:00:00,15760.0,15970.0,195360
4,2025-03-03 00:00:00,15600.0,15770.0,195162
...,...,...,...,...
4291,2008-01-08,28855.0,29135.0,48156
4292,2008-01-07,27905.0,28205.0,48126
4293,2008-01-04,29515.0,29825.0,47784
4294,2008-01-03,28105.0,28400.0,47892


In [ ]:
async def westmetall_async():
    async def get_data(metall):
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        response = requests.get(url=url)
        
        # В ответе лежит вся историческая инф-я по металлам с учетом запасов и 3х
        # месячных фьючерсов
        df = pd.read_html(response.text)
        data = pd.DataFrame()
        # Конкатенируем последний имеющийся год не затирая старые
        data = pd.concat([data, df[0]], axis=0)
        
        # Отечение возможной технической информации/пропусков
        data = data[data["date"] != 'date']
        data = data[data.iloc[:, 1] != '-']
        data = data[data.iloc[:, 2] != '-']
        data = data[data.iloc[:, 3] != '-']
        
        # Препроцессинг типов
        data_preprocessed = data.copy()
        data_preprocessed["date"] = pd.to_datetime(data_preprocessed["date"])
        data_preprocessed.iloc[:, 1] = pd.to_numeric(data_preprocessed.iloc[:, 1])
        data_preprocessed.iloc[:, 2] = pd.to_numeric(data_preprocessed.iloc[:, 2])

        return data_preprocessed

    try:
        # Бачи с новыми данными
        al = await get_data(metall="Al")
        cu = await get_data(metall="Cu")
        pb = await get_data(metall="Pb")
        ni = await get_data(metall="Ni")
        zn = await get_data(metall="Zn")
        tn = await get_data(metall="Sn")

        # Бачи с историческими данными
        old_data_al = pd.read_csv("./data/aluminium_database.csv")
        old_data_cu = pd.read_csv("./data/copper_database.csv")
        old_data_pb = pd.read_csv("./data/lead_database.csv")
        old_data_ni = pd.read_csv("./data/nickel_database.csv")
        old_data_zn = pd.read_csv("./data/zink_database.csv")
        old_data_tn = pd.read_csv("./data/tin_database.csv")
        
        # Склеиваем полученный год дропая дубли
        new_al = pd.concat([al, old_data_al], axis=0)
        new_al.drop_duplicates(inplace=True)
        
        new_cu = pd.concat([cu, old_data_cu], axis=0)
        new_cu.drop_duplicates(inplace=True)
        
        new_pb = pd.concat([pb,old_data_pb], axis=0)
        new_pb.drop_duplicates(inplace=True)
        
        new_ni = pd.concat([ni, old_data_ni], axis=0)
        new_ni.drop_duplicates(inplace=True)
        
        new_zn = pd.concat([zn, old_data_zn], axis=0)
        new_zn.drop_duplicates(inplace=True)    
        
        new_tn = pd.concat([tn, old_data_tn], axis=0)
        new_tn.drop_duplicates(inplace=True)
        
        # Пересохраняем
        new_al.to_csv("./data/aluminium_database.csv", index=False)
        new_cu.to_csv("./data/copper_database.csv", index=False)
        new_pb.to_csv("./data/lead_database.csv", index=False)
        new_ni.to_csv("./data/nickel_database.csv", index=False)
        new_zn.to_csv("./data/zink_database.csv", index=False)
        new_tn.to_csv("./data/tin_database.csv", index=False)

    except Exception as error:
        print(error)
        

if __name__ == "__main__":
    
    print("Parsing starting...")
    
    await asyncio.gather(
        westmetall_async()
    )
    
    print("Parsing complited!!!")

Parsing starting...


In [125]:
import aiohttp
import asyncio
import pandas as pd

async def get_data(metall):
    try:
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                text = await response.text()
                df = pd.read_html(text)
                data = df[0]
                # Обработка данных
                data = data[data["date"] != 'date']
                data = data[data.iloc[:, 1] != '-']
                data = data[data.iloc[:, 2] != '-']
                data = data[data.iloc[:, 3] != '-']
                data["date"] = pd.to_datetime(data["date"])
                data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
                data.iloc[:, 2] = pd.to_numeric(data.iloc[:, 2])
                return data
    except Exception as e:
        print(f"Error fetching data for {metall}: {e}")
        return pd.DataFrame()

async def westmetall_async():
    metals = {
        "Al": "aluminium",
        "Cu": "copper",
        "Pb": "lead",
        "Ni": "nickel",
        "Zn": "zink",
        "Sn": "tin"
    }

    for metall, name in metals.items():
        new_data = await get_data(metall)
        if not new_data.empty:
            old_data = pd.read_csv(f"./data/{name}_database.csv")
            combined_data = pd.concat([new_data, old_data]).drop_duplicates()
            combined_data.to_csv(f"./data/{name}_database.csv", index=False)

if __name__ == "__main__":
    print("Parsing starting...")
    asyncio.run(westmetall_async())
    print("Parsing completed!!!")

Parsing starting...


RuntimeError: asyncio.run() cannot be called from a running event loop

In [128]:
import aiohttp
import asyncio
import pandas as pd

async def get_data(metall):
    try:
        url = f"https://www.westmetall.com/en/markdaten.php?action=table&field=LME_{metall}_cash"
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                text = await response.text()
                df = pd.read_html(text)
                data = df[0]
                # Обработка данных
                data = data[data["date"] != 'date']
                data = data[data.iloc[:, 1] != '-']
                data = data[data.iloc[:, 2] != '-']
                data = data[data.iloc[:, 3] != '-']
                data["date"] = pd.to_datetime(data["date"])
                data.iloc[:, 1] = pd.to_numeric(data.iloc[:, 1])
                data.iloc[:, 2] = pd.to_numeric(data.iloc[:, 2])
                return data
    except Exception as e:
        print(f"Error fetching data for {metall}: {e}")
        return pd.DataFrame()

async def westmetall_async():
    metals = {
        "Al": "aluminium",
        "Cu": "copper",
        "Pb": "lead",
        "Ni": "nickel",
        "Zn": "zink",
        "Sn": "tin"
    }

    for metall, name in metals.items():
        new_data = await get_data(metall)
        if not new_data.empty:
            old_data = pd.read_csv(f"./data/{name}_database.csv")
            combined_data = pd.concat([new_data, old_data]).drop_duplicates()
            combined_data.to_csv(f"./data/{name}_database.csv", index=False)

if __name__ == "__main__":
    print("Parsing starting...")
    
    try:
        # Проверяем, есть ли активный event loop
        loop = asyncio.get_event_loop()
    except RuntimeError:
        # Если нет, создаем новый
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
    
    if loop.is_running():
        # Если event loop уже запущен, используем await
        loop.create_task(westmetall_async())
    else:
        # Если event loop не запущен, используем asyncio.run()
        asyncio.run(westmetall_async())
    
    print("Parsing completed!!!")

Parsing starting...
Parsing completed!!!
